In [ ]:
!pip install transformers datasets --quiet

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
dataset


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Add pad token (GPT-2 has none by default)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


In [10]:
def tokenize_function(example):
    encoding = tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_total_limit=1,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.324300,1.372983
2,1.223800,1.369134
3,1.181800,1.370040


TrainOutput(global_step=13770, training_loss=1.2840192530236558, metrics={'train_runtime': 1637.0565, 'train_samples_per_second': 67.288, 'train_steps_per_second': 8.411, 'total_flos': 3597860283088896.0, 'train_loss': 1.2840192530236558, 'epoch': 3.0})

In [17]:
import torch

input_prompt = "The way to heal mentally is"
input_ids = tokenizer.encode(input_prompt, return_tensors="pt").to("cuda")

# Manually create attention mask since we're padding to max_length
attention_mask = torch.ones_like(input_ids).to("cuda")

# Generate continuation
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id  # important when padding
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


The way to heal mentally is to get people to admit they are not perfect . Many medical students have tried to explain to the world that they are not perfect and that it is not true for them to be perfect , even if they agree with it .


In [18]:
model_path = "distilgpt2-wikitext2"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('distilgpt2-wikitext2/tokenizer_config.json',
 'distilgpt2-wikitext2/special_tokens_map.json',
 'distilgpt2-wikitext2/vocab.json',
 'distilgpt2-wikitext2/merges.txt',
 'distilgpt2-wikitext2/added_tokens.json',
 'distilgpt2-wikitext2/tokenizer.json')

In [19]:
import shutil
shutil.make_archive(model_path, 'zip', model_path)

'/content/distilgpt2-wikitext2.zip'

In [20]:
from google.colab import files
files.download(model_path + ".zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
from huggingface_hub import notebook_login
notebook_login()

model.push_to_hub("Firasat/distilgpt2-wikitext2")
tokenizer.push_to_hub("Firasat/distilgpt2-wikitext2")

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Firasat/distilgpt2-wikitext2/commit/44e1e77f5840f239637e26ce2d59b36ca77197ae', commit_message='Upload tokenizer', commit_description='', oid='44e1e77f5840f239637e26ce2d59b36ca77197ae', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Firasat/distilgpt2-wikitext2', endpoint='https://huggingface.co', repo_type='model', repo_id='Firasat/distilgpt2-wikitext2'), pr_revision=None, pr_num=None)